In [46]:
Project two part 1

# Scrape the content of The Globe and Mail's trending section and save it as a CSV.

# We want: titles, subhead, article URL, byline, article type, image URL.

# Bonus, if you want to get fancy:

#     Make the CSV file auto-updating. Use this tutorial (https://www.youtube.com/watch?v=QNKxzkNpsko) (https://jonathansoma.com/everything/git/auto-updating-scaper-viz/) 


In [47]:
#Columns: title, subhead, article URL, whether it's premium or not, byline, article type, image URL.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.theglobeandmail.com"

#for anonymnity
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36'
}

response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    doc = BeautifulSoup(response.text, 'html.parser')
    # Now you can use soup.find(), soup.find_all(), etc.
else:
    print(f"Request failed with status code: {response.status_code}")


In [2]:
#find all the article in the trending section (there are 8) 
article = doc.find_all('a', class_='c-card__grid c-card__link')
print(f"There are {len(article)} articles in the trending box")

There are 8 articles in the trending box


In [3]:

with open("globe_homepage.html", "w", encoding="utf-8") as f:
    f.write(response.text)

print("HTML saved to globe_homepage.html")

HTML saved to globe_homepage.html


In [ ]:
# #this is the code for a sample chunk in the list of articles that I'm trying to parse 
# <div class="c-card"><a
#                   href="/business/commentary/article-what-was-mark-carney-thinking-when-he-walked-back-the-digital-services/"
#                   class="c-card__grid c-card__link"
#                   data-sophi-label="What was Mark Carney thinking when he walked back the digital services tax?"
#                   data-content-id="D6G7CUTGPZHD3FTL7QTQSU6LHI" data-lt-pos="sec:homepage:trending:recommend-tgam">
#                   <div class="c-card__content"><span
#                       class="StoryLabel__StyledStoryLabel-sc-19mm1gg-0 jZNZzC tgam-label tgam-label--story-default-light text-gml-3"
#                       mode="light">Opinion</span>
#                     <div class="c-card__hed">
#                       <h3 class="c-card__hed-text text-pb-9">What was Mark Carney thinking when he walked back the
#                         digital services tax?</h3>
#                     </div>
#                     <div class="c-card__meta"><span class="c-card__meta__authors"><span
#                           class="c-card__author text-gmr-5">Taylor C. Noakes</span></span></div>
#                   </div>
#                 </a></div>"

In [52]:
# Loop through the eight elements in the list, pulling the information I want 
for rank, link in enumerate(article, start=1):
    title_tag = link.find('h3', class_='c-card__hed-text')
    author_tag = link.find('span', class_='c-card__author')
    label_tag = link.find('span', class_='tgam-label')

    title = title_tag.get_text(strip=True) if title_tag else 'No title'
    author = author_tag.get_text(strip=True) if author_tag else 'No author'
    label = label_tag.get_text(strip=True) if label_tag else 'No label'
    href = link.get('href', '')
    full_url = url + href if href.startswith('/') else href

    print("Rank:", rank)
    print("Title:", title)
    print("Author:", author)
    print("Label:", label)
    print("URL:", full_url)
    print("---")

Rank: 1
Title: In rural B.C., frequent ER closings show how ‘fragile’ the health care system has become
Author: No author
Label: No label
URL: https://www.theglobeandmail.com/canada/article-unplanned-closing-emergency-room-british-columbia-health-care/
---
Rank: 2
Title: What was Mark Carney thinking when he walked back the digital services tax?
Author: Taylor C. Noakes
Label: Opinion
URL: https://www.theglobeandmail.com/business/commentary/article-what-was-mark-carney-thinking-when-he-walked-back-the-digital-services/
---
Rank: 3
Title: Syrian government and leaders of Druze minority announce new ceasefire after Israeli air strikes
Author: No author
Label: No label
URL: https://www.theglobeandmail.com/world/article-israel-strikes-in-damascus-amid-ongoing-clashes-between-government/
---
Rank: 4
Title: ‘The door is now firmly slammed shut on a July BoC rate cut’: Markets and economists react to inflation data
Author: No author
Label: No label
URL: https://www.theglobeandmail.com/investi

In [53]:
rows = []  # List to store all article dicts

for rank, item in enumerate(article, start=1):
    row = {}

    # Title (required)
    title_tag = item.find('h3', class_='c-card__hed-text')
    row['Rank'] = rank 
    row['Article title'] = title_tag.get_text(strip=True) if title_tag else "No title found"

    # URL (required)
    href = item.get('href', '')
    base_url = "https://www.theglobeandmail.com"
    row['url'] = base_url + href if href.startswith('/') else href

    # Byline / Author
    author_tag = item.find('span', class_='c-card__author')
    row['Byline'] = author_tag.get_text(strip=True) if author_tag else "No byline found"

    # Article type / Label (e.g., "Opinion")
    label_tag = item.find('span', class_='tgam-label')
    row['Article type'] = label_tag.get_text(strip=True) if label_tag else "No article type found"

    rows.append(row)

print(f"Total articles scraped: {len(rows)}")


Total articles scraped: 8


In [50]:
df = pd.json_normalize(rows)

In [51]:
df.head(10)

,Rank,Article title,url,Byline,Article type
0,1,"In rural B.C., frequent ER closings show how ‘...",https://www.theglobeandmail.com/canada/article...,No byline found,No article type found
1,2,What was Mark Carney thinking when he walked b...,https://www.theglobeandmail.com/business/comme...,Taylor C. Noakes,Opinion
2,3,Syrian government and leaders of Druze minorit...,https://www.theglobeandmail.com/world/article-...,No byline found,No article type found
3,4,‘The door is now firmly slammed shut on a July...,https://www.theglobeandmail.com/investing/mark...,No byline found,No article type found
4,5,"With Epstein conspiracy theories, Trump faces ...",https://www.theglobeandmail.com/world/article-...,No byline found,No article type found
5,6,RCMP arrest alleged hijacker at Vancouver airp...,https://www.theglobeandmail.com/canada/article...,No byline found,No article type found
6,7,Giller Prize says it will be forced to cease o...,https://www.theglobeandmail.com/arts/books/art...,No byline found,No article type found
7,8,How Australia fell in love with supersized roa...,https://www.theglobeandmail.com/world/article-...,No byline found,No article type found


In [84]:
df.to_csv("GlobeAndMailScrape.csv")